# Introduction

This notebook is used for retrieving product meta data from Edamam API

In [12]:
!pip install openpyxl gensim

import os
import json
import utils
import requests
import pandas as pd
from tqdm import tqdm
from time import sleep

PROJECT_DIR = "."
SOURCES_DIR = os.path.join(PROJECT_DIR, "sources")
DATASETS_DIR = os.path.join(PROJECT_DIR, "datasets")
PRODUCTS_METADATA = os.path.join(SOURCES_DIR, "product_metadata")

os.makedirs(PRODUCTS_METADATA, exist_ok=True)

Defaulting to user installation because normal site-packages is not writeable


In [13]:
nosh_products = utils.read_nosh_products(os.path.join(SOURCES_DIR, "nosh_product_data.xlsx"))

In [14]:
nosh_products

,product_id,product,shelf_life,usual_storage
0,1,spaghetti,NaN,pantry
1,2,onion,13.0,pantry
2,3,sugar,NaN,pantry
3,4,mustard,60.0,pantry
4,5,tomato sauce,NaN,pantry
...,...,...,...,...
202,206,cooked broccoli,7.0,fridge
203,207,mushroom,7.0,fridge
204,208,cooked mushroom,7.0,fridge
205,209,sliced mushroom,5.0,fridge


# Downloading Metadata

The code cell below is used for downloading the metadata relating to each nosh product.

In [2]:
APP_ID = "38278946"
APP_KEY = "9786b96489a752a946c983be874cd290"
ACCESS_POINT = "https://api.edamam.com/api/food-database"

In [6]:
def extract_product_metadata(product):
    """ Extract product information using Edamam API.
    
    Args:
    ---
    - `product`: str
        Product name
    
    Returns:
    ---
    Dict[str,Any]
        A meta data dictionary containing all information relating to `product` from Edamam.
    """
    metadata = {"product": product}
    parser_result = requests.get(
        f"{ACCESS_POINT}/v2/parser?app_id={APP_ID}&app_key={APP_KEY}",
        params={
            'ingr': product
        }
    ).json()
    
    match = parser_result['hints'][0]
    
    metadata['foodId'] = match['food']['foodId']
    metadata['category'] = match['food']['categoryLabel']
    metadata['nutrients'] = match['food']['nutrients']
    metadata['image'] = match['food']['image'] if 'image' in match['food'].keys() else None
    
    metadata['measures'] = match['measures'][0]
    
    ing_response = requests.post(
        f"{ACCESS_POINT}/v2/nutrients?app_id={APP_ID}&app_key={APP_KEY}",
        json={
          "ingredients": [
            {
              "quantity": 1,
              "measureURI": metadata['measures']['uri'],
              "foodId": metadata['foodId']
            }
          ]
        },
        headers={'Content-type': 'application/json', 'Accept': 'application/json'}
    ).json()
    
    metadata['health'] = ing_response
    
    return metadata

In [8]:
for idx, row in tqdm(list(nosh_products.iterrows())):
    filepath = os.path.join(PRODUCTS_METADATA, f"{row['product_id']}.json")
    
    print(f"product: {row['product_id']}")
    
    if os.path.exists(filepath):
        print(f"-- exist, skipped")
    else:
        metadata = extract_product_metadata(row['product'])
        metadata['product_id'] = row['product_id']
        
        if metadata['image'] == None:
            print(f"-- does not have image")
        
        with open(filepath, 'w') as outfile:
            outfile.write(json.dumps(metadata))
        
        sleep(1)

  0%|                                                                                          | 0/207 [00:00<?, ?it/s]

product: 1


  0%|▍                                                                                 | 1/207 [00:01<06:00,  1.75s/it]

product: 2


  1%|▊                                                                                 | 2/207 [00:03<06:34,  1.92s/it]

...

 99%|███████████████████████████████████████████████████████████████████████████████▏| 205/207 [06:13<00:04,  2.02s/it]

product: 209


100%|███████████████████████████████████████████████████████████████████████████████▌| 206/207 [06:15<00:01,  1.94s/it]

product: 210


100%|████████████████████████████████████████████████████████████████████████████████| 207/207 [06:17<00:00,  1.82s/it]


# Merging metadata to nosh products

In [18]:
def read_metadata_json(filepath):
    """ Read metadata json into a pandas DataFrame.
    
    Args:
    ---
    - `filepath`: str
        Path to meta data json file.
        
    Returns:
    ---
    DataFrame
        A DataFrame representing the a metadata.
    """
    with open(filepath, 'r') as fp:
        metadata = json.load(fp)
        
    for k, v in metadata['nutrients'].items():
        metadata[f"nutrients.{k}"] = v
        
    for k, v in metadata['measures'].items():
        metadata[f"measures.{k}"] = v
        
    del metadata['nutrients']
    del metadata['measures']
    
    if 'measures.qualified' in metadata:
        del metadata['measures.qualified']
        
    del metadata['health']['totalNutrients']
    del metadata['health']['totalDaily']
    del metadata['health']['ingredients']
    
    metadata[f"health.healthLabels"] = [str(i) for i in metadata['health']['healthLabels']]
    
    del metadata['health']['healthLabels']
    
    for k, v in metadata['health'].items():
        metadata[f"health.{k}"] = v
        
    del metadata['health']
    del metadata['health.uri']
    del metadata['measures.uri']
    del metadata['foodId']
    del metadata['product']
    
    return pd.DataFrame.from_dict(metadata, orient='index').transpose()

def build_metadata_dataframe(nosh_products):
    """ Read all product metadata into 1 DataFrame.
    
    Args:
    ---
    - `nosh_products`: DataFrame
        The nosh product DataFrame.
        
    Returns:
    ---
    DataFrame
        A metadata DataFrame.
    """
    results = []
    
    for idx, row in nosh_products.iterrows():
        metadata = read_metadata_json(os.path.join(PRODUCTS_METADATA, f"{row['product_id']}.json"))
        results.append(metadata)
    
    return pd.concat(results, axis=0).reset_index(drop=True)

In [19]:
metadata = build_metadata_dataframe(nosh_products)
products = nosh_products.merge(
    right=metadata,
    left_on='product_id',
    right_on='product_id',
    how='left',
)

In [20]:
products.head()

,product_id,product,shelf_life,usual_storage,category,image,nutrients.ENERC_KCAL,nutrients.PROCNT,nutrients.FAT,nutrients.CHOCDF,nutrients.FIBTG,measures.label,measures.weight,health.healthLabels,health.calories,health.totalWeight,health.dietLabels,health.cautions
0,1,spaghetti,NaN,pantry,food,https://www.edamam.com/food-img/296/296ff2b02e...,371.0,13.04,1.51,74.67,3.2,Whole,16.8,"[FAT_FREE, KETO_FRIENDLY, VEGAN, VEGETARIAN, P...",62,16.8,[],[]
1,2,onion,13.0,pantry,food,https://www.edamam.com/food-img/205/205e6bf239...,40.0,1.1,0.1,9.34,1.7,Whole,125.0,"[FAT_FREE, KETO_FRIENDLY, VEGAN, VEGETARIAN, P...",50,125.0,[],[]
2,3,sugar,NaN,pantry,food,https://www.edamam.com/food-img/ecb/ecb3f5aaed...,387.0,0.0,0.0,99.98,0.0,Cube,4.2,"[FAT_FREE, KETO_FRIENDLY, VEGAN, VEGETARIAN, P...",16,4.2,[],[]
3,4,mustard,60.0,pantry,food,https://www.edamam.com/food-img/e23/e238f2e4cf...,60.0,3.74,3.34,5.83,4.0,Serving,30.0,"[KETO_FRIENDLY, VEGAN, VEGETARIAN, PESCATARIAN...",18,30.0,[],[]
4,5,tomato sauce,NaN,pantry,food,https://www.edamam.com/food-img/23e/23e727a14f...,18.0,0.88,0.2,3.89,1.2,Whole,123.0,"[FAT_FREE, KETO_FRIENDLY, VEGAN, VEGETARIAN, P...",22,123.0,[],[]


In [21]:
products.to_csv(os.path.join(DATASETS_DIR, "products.csv"), index=False)

# Link Recipe with Nosh Products

In [7]:
def read_recipe_data(filepath):
    tqdm.pandas()
    df = pd.read_json(filepath)
    df['ingredients'] = df['ingredients'].progress_apply(lambda x: [utils.match_closest_nosh_product(p, nosh_products)['nosh_product_id'] for p in x])
    return df

In [8]:
recipes = read_recipe_data(os.path.join(SOURCES_DIR, "recipes.json")) 

100%|██████████████████████████████████████████████████████████████████████████| 39774/39774 [2:25:05<00:00,  4.57it/s]


In [9]:
recipes.head()

,id,cuisine,ingredients
0,10259,greek,"[72, 192, 72, 75, 85, 2, 87, 61, 139]"
1,25693,southern_us,"[6, 85, 84, 72, 85, 136, 99, 133, 61, 141, 49]"
2,20130,filipino,"[99, 85, 84, 131, 49, 133, 113, 126, 2, 7, 97,..."
3,22213,indian,"[98, 49, 48, 84]"
4,13162,indian,"[85, 91, 34, 85, 2, 149, 141, 97, 84, 92, 98, ..."


In [10]:
recipes.sort_values(by=['id'])\
    .drop('ingredients', axis=1)\
    .to_csv(os.path.join(DATASETS_DIR, 'recipes.csv'), index=False)

In [11]:
recipes.sort_values(by=['id'])\
    .explode('ingredients')\
    .drop('cuisine', axis=1)\
    .rename(columns={"ingredients":"product_id", "id":"recipe_id"})\
    .to_csv(os.path.join(DATASETS_DIR, 'products_recipes.csv'), index=False)